In [1]:
import torch
print(torch.__version__)

2.1.0


In [2]:
import cv2
import numpy as np
from mtcnn import MTCNN
from deep_sort_realtime.deepsort_tracker import DeepSort
import face_recognition
import os
import pickle

In [3]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=10)

In [4]:
# Load pre-trained MTCNN model for face detection
mtcnn = MTCNN()

In [5]:
# Open video file
video_path = "Class Room Entrance.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [6]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for mp4 format
output_path = "output_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

In [7]:
frame_id = 0
skip_frames = 2  # Skip the frames

In [8]:
# Size threshold for faces (adjust as needed)
min_face_size = 10000  # For example, minimum area of 10000 pixels

In [9]:
# Dictionary to track whether the face image and embedding have been saved for each ID
saved_data = {}

In [10]:
# Create a directory to save face images
os.makedirs("faces(V2)", exist_ok=True)

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # Skip frames until reaching the nth frame
    if frame_id <= skip_frames:
        continue

    # Detect faces using MTCNN
    faces = mtcnn.detect_faces(frame)
    bbs = [(face['box'], face['confidence'], face['keypoints']) for face in faces]

    # Update tracker with the detected faces
    tracks = tracker.update_tracks(bbs, frame=frame)

    # Draw bounding boxes and IDs on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id

        # Check if data for this ID has already been saved
        if track_id in saved_data:
            continue

        ltrb = track.to_ltrb()

        # Check face size
        face_area = (ltrb[2] - ltrb[0]) * (ltrb[3] - ltrb[1])
        if face_area < min_face_size:
            continue  # Skip small faces

        # Draw bounding box
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)

        # Get facial landmarks using face_recognition library
        face_locations = [(int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0]))]  # (top, right, bottom, left)
        landmarks = face_recognition.face_landmarks(frame, face_locations)

        if landmarks:
            # Get face embedding
            face_encoding = face_recognition.face_encodings(frame, face_locations)[0]

            # Check if the face embedding is not empty before saving
            if len(face_encoding) > 0:
                # Save the face embedding
                saved_data[track_id] = {
                    "embedding": face_encoding,
                    "image_path": f"faces(V2)/{track_id}_frame{frame_id}.png"
                }

                # Get the face bounding box coordinates
                top, right, bottom, left = int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0])

                # Check if the bounding box coordinates are within the frame boundaries
                if 0 <= top < frame.shape[0] and 0 <= left < frame.shape[1] and top < bottom and left < right:
                    # Capture the face image
                    face_image = frame[top:bottom, left:right]

                    # Check if the face image is not empty before saving
                    if not face_image.size == 0:
                        # Save the face image
                        cv2.imwrite(saved_data[track_id]["image_path"], face_image)

                        # Draw label
                        label = f"ID: {track_id}"
                        cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
    # Write the frame with bounding boxes to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 49ms/step


In [12]:
saved_data

{'1': {'embedding': array([-0.08536583,  0.06535152,  0.07821199,  0.03570478, -0.02554927,
         -0.0623179 ,  0.05149169, -0.17706679,  0.12856854, -0.09910036,
          0.20829074, -0.01849154, -0.20497513, -0.17739913,  0.03787798,
          0.06844736, -0.14030731, -0.16889369,  0.02040816, -0.10481842,
          0.03534599,  0.03289837,  0.02853457,  0.05090513, -0.15343626,
         -0.30458853, -0.07940461, -0.13767505,  0.07229739,  0.00522791,
          0.03056137,  0.11325421, -0.20235789,  0.00731264, -0.0557314 ,
          0.07616745,  0.02971524, -0.01529634,  0.16994558,  0.01935169,
         -0.17999768, -0.08063873, -0.0592166 ,  0.27781171,  0.17604882,
         -0.01591183,  0.00208604,  0.04175801,  0.05632941, -0.21072482,
          0.06895854,  0.12601782,  0.11191633,  0.03085978, -0.08173983,
         -0.19011062, -0.04487886,  0.0347744 , -0.24720594,  0.0540419 ,
          0.00979266, -0.18807541, -0.06814566,  0.0208478 ,  0.20143232,
          0.11139864

In [13]:
# Save the saved_data dictionary to a file
output_data_path = "saved_data.pkl"
with open(output_data_path, 'wb') as f:
    pickle.dump(saved_data, f)

In [17]:
import face_recognition
import pickle

# Load the saved_data dictionary from the file
input_data_path = "saved_data.pkl"
with open(input_data_path, 'rb') as f:
    saved_data = pickle.load(f)

# Create a dictionary to store unique faces
unique_faces = {}
unique_ids_count = 0

# Iterate through each saved face embedding
for current_id, current_data in saved_data.items():
    current_embedding = current_data["embedding"]
    
    # Check if the current embedding is unique by comparing with previous embeddings
    is_unique = True
    for other_id, other_data in unique_faces.items():
        other_embedding = other_data["embedding"]
        
        # Compare faces using face_recognition library
        match = face_recognition.compare_faces([other_embedding], current_embedding)[0]
        
        if match:
            is_unique = False
            break  # If a match is found, no need to check further
    
    # If the current embedding is unique, add it to the unique_faces dictionary
    if is_unique:
        unique_faces[current_id] = current_data
        unique_ids_count += 1

# Save the unique_faces dictionary to a file
output_unique_faces_path = "unique_faces.pkl"
with open(output_unique_faces_path, 'wb') as f:
    pickle.dump(unique_faces, f)

# Print the count of unique IDs
print(f"Count of Unique IDs: {unique_ids_count}")


Count of Unique IDs: 8


In [18]:
unique_faces

{'1': {'embedding': array([-0.08536583,  0.06535152,  0.07821199,  0.03570478, -0.02554927,
         -0.0623179 ,  0.05149169, -0.17706679,  0.12856854, -0.09910036,
          0.20829074, -0.01849154, -0.20497513, -0.17739913,  0.03787798,
          0.06844736, -0.14030731, -0.16889369,  0.02040816, -0.10481842,
          0.03534599,  0.03289837,  0.02853457,  0.05090513, -0.15343626,
         -0.30458853, -0.07940461, -0.13767505,  0.07229739,  0.00522791,
          0.03056137,  0.11325421, -0.20235789,  0.00731264, -0.0557314 ,
          0.07616745,  0.02971524, -0.01529634,  0.16994558,  0.01935169,
         -0.17999768, -0.08063873, -0.0592166 ,  0.27781171,  0.17604882,
         -0.01591183,  0.00208604,  0.04175801,  0.05632941, -0.21072482,
          0.06895854,  0.12601782,  0.11191633,  0.03085978, -0.08173983,
         -0.19011062, -0.04487886,  0.0347744 , -0.24720594,  0.0540419 ,
          0.00979266, -0.18807541, -0.06814566,  0.0208478 ,  0.20143232,
          0.11139864